# Bank

In [ ]:
import sage
import pickle
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.bank()

# Feature names and categorical columns (for CatBoost model)
feature_names = df.columns.tolist()[:-1]
categorical_cols = ['Job', 'Marital', 'Education', 'Default', 'Housing',
                    'Loan', 'Contact', 'Month', 'Prev Outcome']
categorical_inds = [feature_names.index(col) for col in categorical_cols]

In [ ]:
# Split data
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=123)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=123)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier

In [ ]:
model = CatBoostClassifier(iterations=100,
                           learning_rate=0.3,
                           depth=10)

model = model.fit(train, Y_train, categorical_inds, eval_set=(val, Y_val),
                  verbose=False)

In [ ]:
with open('trained_models/bank model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Credit

In [ ]:
import sage
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.credit()

# Feature names and categorical columns (for CatBoost model)
feature_names = df.columns.tolist()[:-1]
categorical_columns = [
    'Checking Status', 'Credit History', 'Purpose', 'Credit Amount',
    'Savings Account/Bonds', 'Employment Since', 'Personal Status',
    'Debtors/Guarantors', 'Property Type', 'Other Installment Plans',
    'Housing Ownership', 'Job', 'Telephone', 'Foreign Worker'
]
categorical_inds = [feature_names.index(col) for col in categorical_columns]

In [ ]:
# Split data
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=0)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=0)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

In [ ]:
import numpy as np
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier

In [ ]:
model = CatBoostClassifier(iterations=50,
                           learning_rate=0.3,
                           depth=3)

model = model.fit(train, Y_train, categorical_inds, eval_set=(val, Y_val),
                  verbose=False)

In [ ]:
with open('trained_models/credit model.pkl', 'wb') as f:
    pickle.dump(model, f)

# MNIST

In [ ]:
import torch
import torchvision.datasets as dsets

In [ ]:
# Load train set
train = dsets.MNIST('../data', train=True, download=True)
imgs = train.data.reshape(-1, 784) / 255.0
labels = train.targets

# Shuffle and split into train and val
inds = torch.randperm(len(train))
imgs = imgs[inds]
labels = labels[inds]
val, Y_val = imgs[:6000], labels[:6000]
train, Y_train = imgs[6000:], labels[6000:]

# Load test set
test = dsets.MNIST('../data', train=False, download=True)
test, Y_test = test.data.reshape(-1, 784) / 255.0, test.targets

# Move test data to numpy
test_np = test.cpu().data.numpy()
Y_test_np = Y_test.cpu().data.numpy()

In [ ]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Create model
device = torch.device('cuda', 3)
model = nn.Sequential(
    nn.Linear(784, 256),
    nn.ELU(),
    nn.Linear(256, 256),
    nn.ELU(),
    nn.Linear(256, 10)).to(device)

# Training parameters
lr = 1e-3
mbsize = 64
max_nepochs = 250
loss_fn = nn.CrossEntropyLoss()
lookback = 5
verbose = False

# Move to GPU
train = train.to(device)
val = val.to(device)
test = test.to(device)
Y_train = Y_train.to(device)
Y_val = Y_val.to(device)
Y_test = Y_test.to(device)

# Data loader
train_set = TensorDataset(train, Y_train)
train_loader = DataLoader(train_set, batch_size=mbsize, shuffle=True)

# Setup
optimizer = optim.Adam(model.parameters(), lr=lr)
min_criterion = np.inf
min_epoch = 0

# Train
for epoch in range(max_nepochs):
    for x, y in train_loader:
        # Move to device.
        x = x.to(device=device)
        y = y.to(device=device)

        # Take gradient step.
        loss = loss_fn(model(x), y)
        loss.backward()
        optimizer.step()
        model.zero_grad()

    # Check progress.
    with torch.no_grad():
        # Calculate validation loss.
        val_loss = loss_fn(model(val), Y_val).item()
        if verbose:
            print('{}Epoch = {}{}'.format('-' * 10, epoch + 1, '-' * 10))
            print('Val loss = {:.4f}'.format(val_loss))

        # Check convergence criterion.
        if val_loss < min_criterion:
            min_criterion = val_loss
            min_epoch = epoch
            best_model = deepcopy(model)
        elif (epoch - min_epoch) == lookback:
            if verbose:
                print('Stopping early')
            break

# Keep best model
model = best_model

In [ ]:
model = model.cpu()
torch.save(model, 'trained_models/mnist mlp.pt')

# Bike

In [ ]:
import sage
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.bike()
feature_names = df.columns.tolist()[:-3]

In [ ]:
# Split data, with total count serving as regression target
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=123)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=123)
Y_train = train[:, -1].copy()
Y_val = val[:, -1].copy()
Y_test = test[:, -1].copy()
train = train[:, :-3].copy()
val = val[:, :-3].copy()
test = test[:, :-3].copy()

In [ ]:
import xgboost as xgb

In [ ]:
# Set up data
dtrain = xgb.DMatrix(train, label=Y_train)
dval = xgb.DMatrix(val, label=Y_val)

# Parameters
param = {
    'max_depth' : 10,
    'objective': 'reg:squarederror',
    'nthread': 4
}
evallist = [(dtrain, 'train'), (dval, 'val')]
num_round = 50

# Train
model = xgb.train(param, dtrain, num_round, evallist, verbose_eval=False)

In [ ]:
with open('trained_models/bike model.pkl', 'wb') as f:
    pickle.dump(model, f)

# BRCA

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
gene_names = [
    'BCL11A', 'IGF1R', 'CCND1', 'CDK6', 'BRCA1', 'BRCA2', 'EZH2', 'SFTPD',
    'CDC5L', 'ADMR', 'TSPAN2', 'EIF5B', 'ADRA2C', 'MRCL3', 'CCDC69', 'ADCY4',
    'TEX14', 'RRM2B', 'SLC22A5', 'HRH1', 'SLC25A1', 'CEBPE', 'IWS1', 'FLJ10213',
    'PSMD10', 'MARCH6', 'PDLIM4', 'SNTB1', 'CHCHD1', 'SCMH1', 'FLJ20489',
    'MDP-1', 'FLJ30092', 'YTHDC2', 'LFNG', 'HOXD10', 'RPS6KA5', 'WDR40B',
    'CST9L', 'ISLR', 'TMBIM1', 'TRABD', 'ARHGAP29', 'C15orf29', 'SCAMP4',
    'TTC31', 'ZNF570', 'RAB42', 'SERPINI2', 'C9orf21'
]

In [ ]:
# Load data.
expression = pd.read_table('data/BRCA_TCGA_microarray.txt',
                           sep='\t', header=0,
                           skiprows=lambda x: x == 1, index_col=0).T
expression.index = pd.Index(
    ['.'.join(sample.split('-')[:3]) for sample in expression.index])

# Filter for reduced gene setif reduced:
expression = expression[gene_names]

# Impute missing values.
expression = expression.fillna(expression.mean())

# Load labels.
labels = pd.read_table('data/TCGA_breast_type.tsv',
                       sep='\t', header=None,
                       index_col=0, names=['Sample', 'Label'])

# Filter for common samples.
expression_index = expression.index.values
labels_index = labels.index.values
intersection = np.intersect1d(expression_index, labels_index)
expression = expression.iloc[[i for i in range(len(expression))
                              if expression_index[i] in intersection]]
labels = labels.iloc[[i for i in range(len(labels))
                      if labels_index[i] in intersection]]

# Join expression data with labels.
label_data = labels['Label'].values
label_index = list(labels.index)
expression['Label'] = np.array(
    [label_data[label_index.index(sample)] for sample in expression.index])
expression['Label'] = pd.Categorical(expression['Label']).codes
data = expression.values

# Split data
train, test = train_test_split(
    data, test_size=int(0.2 * len(data)), random_state=0)
train, val = train_test_split(
    train, test_size=int(0.2 * len(data)), random_state=0)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

# Preprocess
mean = train.mean(axis=0)
std = train.std(axis=0)
train = (train - mean) / std
val = (val - mean) / std
test = (test - mean) / std

In [ ]:
import pickle
import numpy as np
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

In [ ]:
def fit_logistic_regression(train, Y_train, val, Y_val):
    # Tune logistic regression model
    C_list = np.arange(0.1, 1.0, 0.1)
    best_loss = np.inf
    best_C = None

    for C in C_list:
        # Fit model
        model = LogisticRegression(C=C, penalty='l1', multi_class='multinomial',
                                   solver='saga', max_iter=10000)
        model.fit(train, Y_train)

        # Calculate loss
        train_loss = log_loss(Y_train, model.predict_proba(train))
        val_loss = log_loss(Y_val, model.predict_proba(val))
        # print('Train loss = {:.4f}, Val loss = {:.4f}'.format(train_loss, val_loss))

        # See if best
        if val_loss < best_loss:
            best_loss = val_loss
            best_C = C
            
    # Fit model on combined data
    model = LogisticRegression(C=best_C, penalty='l1', multi_class='multinomial',
                               solver='saga', max_iter=10000)
    model.fit(np.concatenate((train, val), axis=0),
              np.concatenate((Y_train, Y_val), axis=0))
    
    return model

In [ ]:
model = fit_logistic_regression(train, Y_train, val, Y_val)

In [ ]:
with open('trained_models/brca model.pkl', 'wb') as f:
    pickle.dump(model, f)